In [1]:
import pandas as pd
import numpy as np
import os

from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urlparse
from selenium.webdriver import Chrome

import tiktoken
import time

In [2]:
urls = ['https://portal.ifba.edu.br/','https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-integrados---','https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-integrados-proeja---','https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-subsequentes---','https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-concomitantes---','https://portal.ifba.edu.br/campi/escolhacampus','https://portal.ifba.edu.br/dgcom/camacari/institucional/contato','https://portal.ifba.edu.br/dgcom/camacari/administrativo/cap']

domain = "portal.ifba.edu.br"

url = "https://portal.ifba.edu.br/"

In [4]:
def get_text_from_url(url):
    '''
        Essa função recebe uma url e faz o scraping do texto da página
        Retorna o texto da página e salva o texto em um arquivo <url>.txt
    '''

    # Analisa a URL e pega o domínio
    local_domain = urlparse(url).netloc
    print(local_domain)

    # Fila para armazenar as urls para fazer o scraping
    fila = deque(urls)
    print(fila)

    # Criar um diretório para armazenar os arquivos de texto
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # Enquanto a fila não estiver vazia, continue fazendo o scraping
    while fila:
        # Pega a próxima URL da fila
        url = fila.pop()
        print("Próxima url",url) # Checa próxima url

        # Salva o texto da url em um arquivo <url>.txt
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding = 'utf8') as f:
            driver = Chrome()
            driver.get(url)
            page_soup = BeautifulSoup(driver.page_source, 'html.parser')
            text = page_soup.get_text()
            f.write(text)

        driver.close()

portal.ifba.edu.br
deque(['https://portal.ifba.edu.br/', 'https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-integrados---', 'https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-integrados-proeja---', 'https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-subsequentes---', 'https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-concomitantes---', 'https://portal.ifba.edu.br/campi/escolhacampus', 'https://portal.ifba.edu.br/dgcom/camacari/institucional/contato', 'https://portal.ifba.edu.br/dgcom/camacari/administrativo/cap'])
Próxima url https://portal.ifba.edu.br/dgcom/camacari/administrativo/cap
Próxima url https://portal.ifba.edu.br/dgcom/camacari/institucional/contato
Próxima url https://portal.ifba.edu.br/campi/escolhacampus
Próxima url https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-concomitantes---
Próxima url https://portal.ifba.edu.br/ensino/nossos-cursos/curso-tecnico/tecnicos-subseq

In [5]:
def remove_newlines(serie):
    '''
        Essa função recebe uma série e remove as quebras de linha
    '''
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

In [6]:
texts=[]
# Obter todos os arquivos de texto no diretório de texto
for file in os.listdir("text/" + domain + "/"):
    # Abra o arquivo e leia o texto
    with open("text/" + domain + "/" + file, "r") as f:
        text = f.read()
        # Omita as primeiras 20 linhas e as últimas 4 linhas e, em seguida, substitua  -, _, e #update com espaços.
        texts.append((file[20:-4].replace('-',' ').replace('_', ' ').replace('#update',''), text))

#Criar um Dataframe a partir da lista de textos
df = pd.DataFrame(texts, columns = ['page_name', 'text'])

# Defina a coluna de texto para ser o texto bruto com as novas linhas removidas
df['text'] = df.page_name + ". " + remove_newlines(df.text)
df.to_csv('processed/scraped.csv')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 846: character maps to <undefined>